In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
import warnings
import math

class SystemModel:
    def __init__(self, config):
        # System parameters from config
        self.service_zone_x = config.get('service_zone_x', 500)
        self.service_zone_y = config.get('service_zone_y', 500)
        self.height_limit = config.get('height_limit', 150)
        self.num_uavs = config.get('num_uavs', 3)
        self.users_per_cell = config.get('users_per_cell', 2)
        self.num_users = self.num_uavs * self.users_per_cell
        self.frequency = config.get('frequency', 2)  # GHz
        self.bandwidth = config.get('bandwidth', 30)  # kHz
        self.r_require = config.get('r_require', 0.15)  # kb
        self.uav_speed = config.get('uav_speed', 5)  # m/s
        self.power_unit = config.get('power_unit', 100 * 10000)  # 20mW
        self.noise_power = config.get('noise_power', 10**(-9) * 10000)
        
        # Initialize positions
        self.initialize_positions()
        
        # Initialize state space
        self.state = np.zeros([1, self.num_uavs * 3 + self.num_users], dtype=float)
        
        # Initialize power allocation
        self.power_allocation = pd.DataFrame(
            np.ones((1, self.num_users)) * self.power_unit,
            columns=np.arange(self.num_users).tolist()
        )
        
    def initialize_positions(self):
        """Initialize UAV and user positions"""
        # Initialize UAVs
        self.uav_positions = pd.DataFrame(
            np.zeros((3, self.num_uavs)),
            columns=np.arange(self.num_uavs).tolist()
        )
        
        # Set initial UAV positions
        self.uav_positions.iloc[0, :] = [100, 200, 400]  # x coordinates
        self.uav_positions.iloc[1, :] = [100, 400, 400]  # y coordinates
        self.uav_positions.iloc[2, :] = [100, 100, 100]  # z coordinates
        
        # Initialize users
        self.user_positions = pd.DataFrame(
            np.zeros((3, self.num_users)),
            columns=np.arange(self.num_users).tolist()
        )
        
        # Set initial user positions
        self.user_positions.iloc[0, :] = [162, 176, 227, 383, 458, 43]
        self.user_positions.iloc[1, :] = [117, 62, 183, 426, 397, 328]
        self.user_positions.iloc[2, :] = 0  # Ground level
        
        # Store initial positions for reset
        self.initial_uav_positions = self.uav_positions.copy()
        self.initial_user_positions = self.user_positions.copy()

    def calculate_channel_gain(self, uav_idx, user_idx):
        """Calculate channel gain between UAV and user with LoS probability"""
        uav_pos = self.uav_positions.iloc[:, uav_idx]
        user_pos = self.user_positions.iloc[:, user_idx]
        
        # Calculate 3D distance
        distance = np.linalg.norm(uav_pos - user_pos)
        
        # Calculate LoS probability
        height = uav_pos[2]
        d_2d = np.sqrt(distance**2 - height**2)
        theta = np.arctan(height/d_2d)
        p_los = 1 / (1 + self.a * np.exp(-self.b * (theta - self.a)))
        
        # Calculate path loss
        pl_los = self.calculate_path_loss_los(distance, self.frequency)
        pl_nlos = self.calculate_path_loss_nlos(distance, self.frequency)
        
        # Combined path loss
        path_loss = p_los * pl_los + (1 - p_los) * pl_nlos
        
        # Apply small-scale fading
        fading = np.random.rayleigh(scale=1.0)
        
        return path_loss * fading

    def calculate_sinr(self, user_association):
        """Calculate SINR for all users with NOMA power allocation"""
        sinr = pd.DataFrame(
            np.zeros((1, self.num_users)),
            columns=np.arange(self.num_users).tolist()
        )
        
        for user_idx in range(self.num_users):
            serving_uav = user_association.iloc[0, user_idx]
            signal_power = (self.power_allocation.iloc[0, user_idx] * 
                          self.calculate_channel_gain(serving_uav, user_idx))
            
            interference = 0
            
            # Intra-cell interference (NOMA)
            cluster_users = np.where(user_association.iloc[0, :] == serving_uav)[0]
            for other_user in cluster_users:
                if other_user != user_idx:
                    channel_gain_user = self.calculate_channel_gain(serving_uav, user_idx)
                    channel_gain_other = self.calculate_channel_gain(serving_uav, other_user)
                    if channel_gain_user < channel_gain_other:
                        interference += (self.power_allocation.iloc[0, other_user] * 
                                      self.calculate_channel_gain(serving_uav, user_idx))
            
            # Inter-cell interference
            for uav_idx in range(self.num_uavs):
                if uav_idx != serving_uav:
                    interference += np.sum([
                        self.power_allocation.iloc[0, u] * 
                        self.calculate_channel_gain(uav_idx, user_idx)
                        for u in np.where(user_association.iloc[0, :] == uav_idx)[0]
                    ])
            
            sinr.iloc[0, user_idx] = signal_power / (interference + self.noise_power)
            
        return sinr

    def calculate_rates(self, sinr):
        """Calculate data rates for all users"""
        rates = pd.DataFrame(
            np.zeros((1, self.num_users)),
            columns=np.arange(self.num_users).tolist()
        )
        
        for user_idx in range(self.num_users):
            rates.iloc[0, user_idx] = (self.bandwidth * 
                                     math.log2(1 + sinr.iloc[0, user_idx]))
        
        sum_rate = np.sum(rates.values)
        worst_rate = np.min(rates.values)
        
        return rates, sum_rate, worst_rate

    def take_action(self, action, uav_idx, power_allocation):
        """Execute UAV movement and power allocation action"""
        # Extract movement and power components
        movement = action % 7
        
        # Update UAV position based on movement
        if movement == 0:  # move right
            self.uav_positions.iloc[0, uav_idx] = min(
                self.uav_positions.iloc[0, uav_idx] + self.uav_speed,
                self.service_zone_x
            )
        elif movement == 1:  # move left
            self.uav_positions.iloc[0, uav_idx] = max(
                self.uav_positions.iloc[0, uav_idx] - self.uav_speed,
                0
            )
        elif movement == 2:  # move forward
            self.uav_positions.iloc[1, uav_idx] = min(
                self.uav_positions.iloc[1, uav_idx] + self.uav_speed,
                self.service_zone_y
            )
        elif movement == 3:  # move backward
            self.uav_positions.iloc[1, uav_idx] = max(
                self.uav_positions.iloc[1, uav_idx] - self.uav_speed,
                0
            )
        elif movement == 4:  # move up
            self.uav_positions.iloc[2, uav_idx] = min(
                self.uav_positions.iloc[2, uav_idx] + self.uav_speed,
                self.height_limit
            )
        elif movement == 5:  # move down
            self.uav_positions.iloc[2, uav_idx] = max(
                self.uav_positions.iloc[2, uav_idx] - self.uav_speed,
                20
            )
        # movement == 6 is hover, no position update needed
        
        # Update power allocation
        for user, power in power_allocation.items():
            self.power_allocation.iloc[0, user] = power

    def get_state(self, serving_uav):
        """Get current state representation"""
        state = np.zeros(self.num_uavs * 3 + self.num_users)
        
        # UAV positions
        for uav_idx in range(self.num_uavs):
            state[uav_idx * 3:(uav_idx + 1) * 3] = self.uav_positions.iloc[:, uav_idx]
        
        # Channel gains
        for user_idx in range(self.num_users):
            state[self.num_uavs * 3 + user_idx] = self.calculate_channel_gain(
                serving_uav, user_idx
            )
        
        return state

    def reset(self):
        """Reset environment to initial state"""
        self.uav_positions = self.initial_uav_positions.copy()
        self.user_positions = self.initial_user_positions.copy()
        self.power_allocation = pd.DataFrame(
            np.ones((1, self.num_users)) * self.power_unit,
            columns=np.arange(self.num_users).tolist()
        )
        return self.get_state(0)  # Return state for first UAV